# preamble

In [1]:
# !pip install -r requirements.txt -q

In [1]:
import requests
import os
import openai

from dotenv import load_dotenv
load_dotenv();

In [2]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
NOTION_API_KEY = os.environ["NOTION_API_KEY"]
NOTION_DATABASE_ID = os.environ["NOTION_DATABASE_ID"]

# receive voicenote

In [28]:
title_voicenote = "230828 research meeting"

# transcribe

In [29]:
voice_note = open(f"./voicenotes/{title_voicenote}.mp3", "rb")
transcript = openai.Audio.transcribe(api_key=OPENAI_API_KEY,
                                     model="whisper-1",
                                     file=voice_note,
                                     response_format="text",
                                     language="en"
                                     )

transcript

### split long files

In [11]:
# not needed when also 16k context model can be used

# max_tokens = 4097
# avg_token_len = 4
# max_trans_len = max_tokens * avg_token_len - 100 #for safety

# len(transcript) / max_trans_len

2.4601547151277012

# generate summary, topic list, and critical questions

In [4]:
# summary_prompt      = """\
# You are a helpful assistant, specialized in summarizing and understanding the core of a given message. \
# For the given piece of text, you are going to perform the following four tasks. \
# Firstly, start your response with a consice title describing the text. \
# Secondly, summarize in first person the content of this text in about one alinea. \
# Thirdly, list the different topics discussed in this text. Only add a topic when it is discussed for at least a couple of sentences. \
# Fourth, imagine being a critical life coach of the person writing this text. What are the three best questions you could ask regarding this text?\
# """

In [6]:
summary_prompt      = """\
You are a helpful assistant, specialized in summarizing and understanding the core of a given message. \
For the given piece of text, you are going to perform the following four tasks. \
Firstly, start your response with a consice title describing the text. \
Secondly, summarize in first person the content of this text in one coherent story of about one alinea written. \
Thirdly, list the different topics discussed in this text. Only add a topic when it is discussed for at least a couple of sentences. \
Fourth, imagine being a critical life coach of the person writing this text. What are the three best questions you could ask regarding this text?\
"""

In [30]:
text_response = openai.ChatCompletion.create(api_key=OPENAI_API_KEY,
                                             model="gpt-3.5-turbo",
                                             messages=[
                                                 {"role": "system", "content": summary_prompt},
                                                 {"role": "user", "content": transcript}
                                             ])

In [31]:
text_response["choices"][0]["message"]["content"]

'Meeting Recap with Gunther Nuzai: Need for a Well-Defined Thesis and Ethical Data Set\n\nIn my recent thesis meeting with Gunther Nuzai, I presented my current idea for the thesis and explained my research question, which was met with approval. However, when I introduced my two hypotheses connecting philosophy to computer science, Gunther expressed concern about the lack of clear connection between them. Overall, the main feedback from Gunther was that I seemed to be attempting too much research and becoming too superficial. Gunther emphasized the importance of formalizing and clearly defining my work to avoid this issue. Both Gunther and Nuzai also expressed skepticism about the ethics data set and its relevance. They were unclear about what it entailed and how it related to the research. Gunther advised me to write a well-argued and formalized explanation of my approach and share it with him for better understanding. He stressed the importance of meaningful learning from the thesis 

## process response

In [32]:
text_response["choices"][0]["message"]["content"].split('\n')

['Meeting Recap with Gunther Nuzai: Need for a Well-Defined Thesis and Ethical Data Set',
 '',
 'In my recent thesis meeting with Gunther Nuzai, I presented my current idea for the thesis and explained my research question, which was met with approval. However, when I introduced my two hypotheses connecting philosophy to computer science, Gunther expressed concern about the lack of clear connection between them. Overall, the main feedback from Gunther was that I seemed to be attempting too much research and becoming too superficial. Gunther emphasized the importance of formalizing and clearly defining my work to avoid this issue. Both Gunther and Nuzai also expressed skepticism about the ethics data set and its relevance. They were unclear about what it entailed and how it related to the research. Gunther advised me to write a well-argued and formalized explanation of my approach and share it with him for better understanding. He stressed the importance of meaningful learning from the 

In [33]:
content_notion = text_response["choices"][0]["message"]["content"].split("\n\n")

title_notion = 'Not found'
summary_notion = 'Not found'
topics_notion = 'Not found'
critical_questions_notion = 'Not found'

title_notion = content_notion[0]
summary_notion = content_notion[1]
topics_notion = content_notion[2]
critical_questions_notion = content_notion[3]

In [25]:
# content_notion = text_response["choices"][0]["message"]["content"].split("\n")

# title_notion = 'Not found'
# summary_notion = 'Not found'
# topics_notion = 'Not found'
# critical_questions_notion = 'Not found'

# title_notion = content_notion[0]
# summary_notion = content_notion[1]
# topics_notion = '\n'.join(content_notion[4:10])
# critical_questions_notion = '\n'.join(content_notion[-3:])

In [36]:
summary_notion

'In my recent thesis meeting with Gunther Nuzai, I presented my current idea for the thesis and explained my research question, which was met with approval. However, when I introduced my two hypotheses connecting philosophy to computer science, Gunther expressed concern about the lack of clear connection between them. Overall, the main feedback from Gunther was that I seemed to be attempting too much research and becoming too superficial. Gunther emphasized the importance of formalizing and clearly defining my work to avoid this issue. Both Gunther and Nuzai also expressed skepticism about the ethics data set and its relevance. They were unclear about what it entailed and how it related to the research. Gunther advised me to write a well-argued and formalized explanation of my approach and share it with him for better understanding. He stressed the importance of meaningful learning from the thesis and the need for well-justified choices. Gunther suggested taking baby steps and creating

# create Notion page

In [37]:
url = "https://api.notion.com/v1/pages/"

headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
}

data_database = {

    "parent":   {
        "type": "database_id",
        "database_id": NOTION_DATABASE_ID
                },
    "icon": {"emoji": "🎙️"},

    "properties":   {
        "Name":         {
            "title": [{"text": {"content": title_notion}}]
                        }  
                    },

    "children": [
        {
            "object": "block",
            "heading_2": {"rich_text": [{"text": {"content": "Summary"}}]}
        },
        {
            "object": "block",
            "paragraph": {"rich_text": [{"text": {"content": summary_notion}}], "color": "default"}
        },
        {
            "object": "block",
            "heading_2": {"rich_text": [{"text": {"content": "Topics"}}]}
        },
        {
            "object": "block",
            "paragraph": {"rich_text": [{"text": {"content": topics_notion}}], "color": "default"}
        },
        {
            "object": "block",
            "heading_2": {"rich_text": [{"text": {"content": "Critical Questions"}}]}
        },
        {
            "object": "block",
            "paragraph": {"rich_text": [{"text": {"content": critical_questions_notion}}], "color": "default"}
        },
        {
            "object": "block",
            "heading_2": {"rich_text": [{"text": {"content": "Transcript"}}]}
        },
        {
            "object": "block",
            "paragraph": {"rich_text": [{"text": {"content": transcript}}], "color": "default"}
        },
                ]

}

response_get = requests.post(url, headers=headers, json=data_database)
response_get.json()

{'object': 'page',
 'id': '5f5316be-90dc-4001-a90d-3e855be3adcc',
 'created_time': '2023-08-29T06:28:00.000Z',
 'last_edited_time': '2023-08-29T06:28:00.000Z',
 'created_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'last_edited_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'cover': None,
 'icon': {'type': 'emoji', 'emoji': '🎙️'},
 'parent': {'type': 'database_id',
  'database_id': 'e5568bc0-5015-4ec7-bd8c-ba5050353de6'},
 'archived': False,
 'properties': {'Created time': {'id': 'BAiK',
   'type': 'created_time',
   'created_time': '2023-08-29T06:28:00.000Z'},
  'Name': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text',
     'text': {'content': 'Meeting Recap with Gunther Nuzai: Need for a Well-Defined Thesis and Ethical Data Set',
      'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
  